In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import csv
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM

#Encoder-Decoder Model
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer
from transformers import Trainer
from evaluate import load
#Training

from transformers import TrainingArguments
from transformers import  Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

from datasets import Dataset
import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss

# bertscore = load("bertscore") 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")

[2023-11-02 18:27:25,894] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_labels)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

In [9]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
    print("====================================")
    inpu.extend([text]*10)
    c =[]
    for i, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


  0%|          | 1/10000 [00:00<2:44:34,  1.01it/s]

  0%|          | 2/10000 [00:01<2:10:19,  1.28it/s]

  0%|          | 3/10000 [00:02<2:08:21,  1.30it/s]

  0%|          | 4/10000 [00:02<1:51:13,  1.50it/s]

  0%|          | 5/10000 [00:03<1:51:27,  1.49it/s]

...

  7%|▋         | 738/10000 [11:03<2:08:50,  1.20it/s]

  7%|▋         | 739/10000 [11:04<2:40:51,  1.04s/it]

  7%|▋         | 740/10000 [11:05<2:24:40,  1.07it/s]

  7%|▋         | 741/10000 [11:06<2:33:19,  1.01it/s]

  7%|▋         | 742/10000 [11:07<2:19:16,  1.11it/s]